In [74]:
from boto3.dynamodb.conditions import Key, Attr
from dotenv import load_dotenv
import boto3
import json
import os

INFORMACJE UWIERZYTELNIANIA

In [82]:
load_dotenv(f".env")

TABLE_NAME = "ue_dynamodb_warsztaty"
AWS_ACCESS_KEY = os.environ["AWS_ACCESS_KEY"]
AWS_SECRET_KEY = os.environ["AWS_SECRET_KEY"]
REGION = os.environ["REGION"]

FUNKCJE POMOCNICZE

In [92]:
def print_json(text: str) -> None:
    print(json.dumps(text, indent=2))

NAWIĄZANIE POŁĄCZENIA

In [83]:
# Utworzenie sesji
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=REGION
)
# Utworzenie klienta
client = session.resource('dynamodb')
table = client.Table(TABLE_NAME)

POBIERANIE / ODCZYTYWANIE ZAWARTOŚCI

In [84]:
# Pobranie zawartości tabeli
response = table.scan()
items = response['Items']

In [85]:
# Przetwarzanie pobranych elementów
for item in items:
    print(item)

{'kraj': 'Polska', 'hobby': ['wędrówki', 'gotowanie'], 'imię': 'Piotr', 'sort_key': 'G', 'id': Decimal('7'), 'wiek': Decimal('45')}
{'płeć': 'kobieta', 'imię': 'Olga', 'sort_key': 'H', 'id': Decimal('8'), 'email': 'olga@example.com', 'wiek': Decimal('27')}
{'kraj': 'Niemcy', 'adres': {'miasto': 'Berlin', 'ulica': 'Niemiecka 456'}, 'imię': 'Zofia', 'sort_key': 'J', 'id': Decimal('10'), 'wiek': Decimal('29')}
{'sort_key': 'C', 'id': Decimal('3'), 'zawód': 'Inżynier', 'wiek': Decimal('35'), 'imię': 'Paweł'}
{'sort_key': 'B', 'kraj': 'Polska', 'id': Decimal('2'), 'wiek': Decimal('30'), 'imię': 'Alicja'}
{'hobby': ['malarstwo', 'ogrodnictwo'], 'imię': 'Liliana', 'sort_key': 'N', 'id': Decimal('14'), 'wiek': Decimal('26'), 'edukacja': {'kierunek': 'Sztuki Piękne', 'stopień': 'Licencjat'}}
{'miasto': 'Łódź', 'zainteresowania': ['podróże', 'fotografia'], 'imię': 'Emilia', 'sort_key': 'L', 'id': Decimal('12'), 'wiek': Decimal('31')}
{'telefon': '+48123456789', 'imię': 'Adam', 'sort_key': 'I', '

EQ | NE | LE | LT | GE | GT | NOT_NULL | NULL | CONTAINS | NOT_CONTAINS | BEGINS_WITH | IN | BETWEEN

In [108]:
# Definiowanie wyrażenia filtru
# filter_expression = Key('id').eq(3) & Key('sort_key').eq('C')

filter_expression = Attr('wiek').gt(20) & Attr('dział').begins_with('Sprz')

# Wykonywanie operacji odczytu na tabeli z filtrem
response = table.scan(FilterExpression=filter_expression)
# Przetwarzanie pobranych elementów
items = response['Items']

for item in items:
    print(item)

{'imię': 'Anna', 'sort_key': 'F', 'stanowisko': 'Menadżer', 'id': Decimal('6'), 'wiek': Decimal('32'), 'dział': 'Sprzedaż'}


DODAWANIE WARTOŚCI

In [109]:
# Definiowanie nowego rekordu
item = {
    'id': 16,
    'sort_key': 'A',
    'imię': 'Jakub',
    'adres': 
        {
            'miasto': 'Katowice', 
            'województwo': 'Sląskie', 
            'ulica': 'Warszawska 15'
        }
}

# Dodawanie rekordu do tabeli
response = table.put_item(Item=item)
print_json(response)

{
  "ResponseMetadata": {
    "RequestId": "BDA8JHBM62MHL8U1479SQ2OMMFVV4KQNSO5AEMVJF66Q9ASUAAJG",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "server": "Server",
      "date": "Sat, 10 Jun 2023 06:50:40 GMT",
      "content-type": "application/x-amz-json-1.0",
      "content-length": "2",
      "connection": "keep-alive",
      "x-amzn-requestid": "BDA8JHBM62MHL8U1479SQ2OMMFVV4KQNSO5AEMVJF66Q9ASUAAJG",
      "x-amz-crc32": "2745614147"
    },
    "RetryAttempts": 0
  }
}


EDYCJA ISNIEJĄCEGO REKORDU

In [110]:
key = {
    'id': 16,
    'sort_key': 'A',
}

# Definiowanie wyrażenia aktualizacji
update_expression = 'SET #attr1 = :val1, #attr2 = :val2'

# Definiowanie nazw atrybutów
expression_attribute_names = {
    '#attr1': 'imię',
    '#attr2': 'stanowisko'
}

# Definiowanie wartości dla wyrażenia aktualizacji
expression_attribute_values = {
    ':val1': 'Robert',
    ':val2': 'Mechanik'
}

# Aktualizacja elementu w tabeli
response = table.update_item(
    Key=key,
    UpdateExpression=update_expression,
    ExpressionAttributeNames=expression_attribute_names,
    ExpressionAttributeValues=expression_attribute_values,
)

# Wyświetlanie informacji o odpowiedzi
print_json(response)

{
  "ResponseMetadata": {
    "RequestId": "B0HPBB7KT13PCD2NA7E0NF03P3VV4KQNSO5AEMVJF66Q9ASUAAJG",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "server": "Server",
      "date": "Sat, 10 Jun 2023 06:56:44 GMT",
      "content-type": "application/x-amz-json-1.0",
      "content-length": "2",
      "connection": "keep-alive",
      "x-amzn-requestid": "B0HPBB7KT13PCD2NA7E0NF03P3VV4KQNSO5AEMVJF66Q9ASUAAJG",
      "x-amz-crc32": "2745614147"
    },
    "RetryAttempts": 0
  }
}


USUWANIE REKORDU

In [111]:
# Definiowanie klucza głównego rekordu do usunięcia
key = {
    'id': 16,
    'sort_key': 'A',
}

# Usuwanie rekordu z tabeli
response = table.delete_item(Key=key)

# Wyświetlanie informacji o odpowiedzi
print(json.dumps(response, indent=2))

{
  "ResponseMetadata": {
    "RequestId": "TDTGTN9MCL4S331N9RUCSPF64VVV4KQNSO5AEMVJF66Q9ASUAAJG",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "server": "Server",
      "date": "Sat, 10 Jun 2023 06:57:04 GMT",
      "content-type": "application/x-amz-json-1.0",
      "content-length": "2",
      "connection": "keep-alive",
      "x-amzn-requestid": "TDTGTN9MCL4S331N9RUCSPF64VVV4KQNSO5AEMVJF66Q9ASUAAJG",
      "x-amz-crc32": "2745614147"
    },
    "RetryAttempts": 0
  }
}


TWORZENIE NOWEJ TABELI

In [115]:
# Definiowanie schematu tabeli
table_schema = {
    'TableName': 'ue_tabela_2',
    'KeySchema': [
        {
            'AttributeName': 'partition_key',
            'KeyType': 'HASH'  # Klucz partycji
        },
        {
            'AttributeName': 'sort_key',
            'KeyType': 'RANGE'  # Klucz sortujący
        }
    ],
    'AttributeDefinitions': [
        {
            'AttributeName': 'partition_key',
            'AttributeType': 'S'  # Typ atrybutu dla klucza partycji (np. 'S' dla string)
        },
        {
            'AttributeName': 'sort_key',
            'AttributeType': 'N'  # Typ atrybutu dla klucza sortującego (np. 'N' dla liczby)
        }
    ],
    'ProvisionedThroughput': {
        'ReadCapacityUnits': 5,  # Pojemność odczytu jednostek na sekundę
        'WriteCapacityUnits': 5  # Pojemność zapisu jednostek na sekundę
    }
}

# Utworzenie klienta
client = session.client('dynamodb')

# Tworzenie tabeli
response = client.create_table(**table_schema)

# Wyświetlanie informacji o odpowiedzi
print(response)

TypeError: Object of type datetime is not JSON serializable